In [ ]:
%matplotlib inline
import os
import json
import time
import requests
import datetime
import dateutil
import pandas as pd
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from urllib.request import urlopen
import json
import plotly.express as px
sns.set()

#### Event-based Contagion Forecasting
The following analysis seeks to determine if there is a correlation between the information on Covid-19, number of Covid-19 cases and the number of attributed deaths in relation to the timing and location of Trump's 2020 campaign rallies for U.S. President.  The analysis assumes that the covid statistics will be captured for each city in which a rally occured. The rally schedule ('https://ballotpedia.org/Donald_Trump_presidential_campaign,_2020#Campaign_rallies') is listed by city and New York Times Covid-19 data ('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv') is by county, therefore the table is joined first by looking up the county for each city, and getting the fips coded geographic boundaries:

In [ ]:
cities=pd.read_csv('https://raw.githubusercontent.com/kelvins/US-Cities-Database/main/csv/us_cities.csv'
                  ).drop('STATE_CODE',axis=1)
cities.columns=[i.lower().split('_')[0] for i in cities.columns]

In [ ]:
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
    
county_coordinates=pd.DataFrame([(item['id'],item['geometry']['coordinates'][0][0]) 
                          for item in counties['features']],columns=['fips','coord'])

Covid-19 dataset is downloaded to a dataframe:

In [ ]:
covid=pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv',
                  parse_dates=['date'],dtype={"fips": str}).set_index("date")
covid['county_state']=covid.county+'_'+covid.state
print("The complete Covid-19 dataset has {} records".format(covid.shape[0]))
covid.tail()

In [ ]:
rallies=pd.read_excel('../../../Desktop/trump_rally_spread/'+'trump_rallies.xlsx')
rallies.columns=['date','city','state']
covid_era=[1 for i in range(rallies[rallies.date > pd.Timestamp('2020-03-13')].shape[0])]+[0 for i in range(rallies[rallies.date < pd.Timestamp('2020-03-13')].shape[0])]
rallies['covid_era']=covid_era
print('Trump campaign has conducted {} campaign rallies since March 13, 2020'.format(rallies[rallies.date > pd.Timestamp('2020-03-13')].shape[0]))
print("{} percent of Trump's rallies have taken place during the pandemic.".format(rallies['covid_era'].sum() / rallies['covid_era'].count()*100))

For every rally city find the county the city is in.

In [ ]:
lst=[]
for d,rc,rs in zip(rallies.date,rallies.city,rallies.state):
    try:
        lst.append((d,rc,cities[(cities.city==rc)& (cities.state==rs)]['county'].values[0],rs))
    except:
        lst.append((d,rc,'Check',rs))

In [ ]:
rallies_w_county=pd.DataFrame(lst,columns=['date','city','county','state'])
rallies_w_county.at[27,'county']='Polk'
rallies_w_county['county_state']=rallies_w_county.county+'_'+rallies_w_county.state
rallies_w_county.county_state=rallies_w_county.county_state.replace('Saint Louis_Minnesota', 'St. Louis_Minnesota')
rallies_w_county.county_state=rallies_w_county.county_state.replace('Check_Florida', 'Broward_Florida')
rallies_w_county.county_state=rallies_w_county.county_state.replace('Check_North Carolina', 'Forsyth_North Carolina')
rallies_w_county.county_state=rallies_w_county.county_state.replace('Miami-dade_Florida', 'Miami-Dade_Flordia')
rallies_w_county.head()

 Merge with trump rallies dataset and then **save** it.

We'll calculate basic statistics for the entire covid dataset...

In [ ]:
ROLLING_MEAN_WINDOW=5
df=pd.DataFrame()
start_date='2020-02-01'
dict_={'y':1 , 'n': 0}
lsty=[]
for county_state in list(set(covid.county_state)):
    tmp=covid[(covid.county_state==county_state) & (covid.index > start_date)]
    tmp.loc[:,"new_cases"]=tmp.cases.diff()
    tmp.loc[:,"new_deaths"]=tmp.deaths.diff()
    tmp.loc[:,'cases_pct_change']=tmp.cases.pct_change(periods=ROLLING_MEAN_WINDOW)
    tmp.loc[:,'deaths_pct_change']=tmp.deaths.pct_change(periods=ROLLING_MEAN_WINDOW)
    #print(tmp.shape[0])
    if county_state in list(rallies_w_county.county_state):
        lsty.append(["y"]*(tmp.shape[0]))
    else:
        lsty.append(["n"]*(tmp.shape[0]))
    tmp['y_county']=lsty[0]
    lsty=[]

    df=pd.concat([df,tmp])


df['y_count']=[dict_[i] for i in df['y_county']]
df.merge(county_coordinates, on='fips')
df.to_pickle('trump.pkl')

**load data** from saved copy we don't have to rerun processing

In [ ]:
df=pd.read_pickle('trump.pkl')
df.sample(5)

#### Inspecting the data

In [ ]:
#all_states_only=df.replace([np.inf, -np.inf], np.nan).groupby(['state_x','fips'])[['cases_pct_change']].mean().reset_index()
df=df.replace([np.inf, -np.inf], np.nan)
df.describe()

In [ ]:
#df.groupby(['state_x','date'])[['cases_pct_change']].mean().reset_index().plot()
states=df.groupby('state')[['new_cases','cases_pct_change']].agg({'cases_pct_change': ['min', 'max', 'mean'], 'new_cases': ['sum']})
states.sort_values(('cases_pct_change','max'),ascending=False)

#### Distribution of All States. The mean pct change for all states during the period.

In [ ]:
stat=states['cases_pct_change']
sns.distplot(stat['mean'])
print('{:.02f} percent'.format(stat['mean'].mean()*100))

#### States experiencing the biggest average increase in cases since March 13, 2020

In [ ]:
stat.sort_values('mean', ascending=False).head(10)

#### States experiencing lowest average increase since March 13, 2020.

In [ ]:
stat.sort_values('mean', ascending=False).tail(10)

#### Trump Rallies Since March 13, 2020

In [ ]:
sns.set()
def filter_features(df,source_column,target_item_list):
    return df[df[source_column].isin(target_item_list)]

def filter_time(df,target_date,time_window,look_forward):
    if look_forward:
        delta= target_date+pd.Timedelta(days=time_window)
        tmp=df[(df.index > target_date) & (df.index < delta)]
    else:
        delta= target_date-pd.Timedelta(days=time_window)
        tmp=df[(df.index > delta) & (df.index < target_date)]
    return tmp

def plot_effect(rally,rally_date,show_off,stat):
    """rallies_w_county.county[0]"""
    plt.figure(figsize=(15,4))

    ax=sns.lineplot(data=rally.new_cases, color="y",linewidth = 3);
    ax2 = ax.twinx()
    ax2.bar(rally.index,rally.new_deaths,color='red',alpha=.2)

    ax.set_ylabel('cases',color="y",fontsize=16)
    ax2.set_ylabel('deaths',color="red",fontsize=14)
    
    offset = pd.Timedelta(show_off, 'D')
    
    plt.title('Trump Effect in {} | daily avg. incr. in cases: {:.02f} percent'.format(rally.county_state[0], stat*100))
    #ax.legend(['input'], loc=2)

    ax.axvline(rally_date + offset, alpha=0.3, color='black')
    ax.axvline(rally_date, alpha=0.9, color='red', linewidth=2.5)
    ax.axvline(rally_date - offset, alpha=0.3, color='black')
    ymark=rally.new_cases.max()
    ax.text(x=rally_date, y=int(ymark*.6), s='Trump Rally \n{:%Y-%m-%d}'.format(rally_date), color='r',fontsize=15)
    ax.text(x=rally_date+offset, y=int(ymark*.4), s='{} Days After'.format(show_off), alpha=.5, color='b')
    ax.text(x=rally_date-offset, y=int(ymark*.4), s='{} Days Before'.format(show_off), alpha=.5, color='b')
    
    return plt

In [ ]:
TIME_WINDOW=20

def trump_rally_effect(select,stat=False,plot=False,verbose=False):    
    selection=list(rallies_w_county[['date','county_state']].iloc[select])
    selected_county_state=filter_features(df,'county_state',[selection[1]])
    look_forward=filter_time(selected_county_state,selection[0],TIME_WINDOW,True)
    look_backward=filter_time(selected_county_state,selection[0],TIME_WINDOW,False)
    look_span=pd.concat([look_backward, look_forward])
    backward_span=filter_time(selected_county_state,selection[0],20,False)
    view_span=pd.concat([backward_span, look_forward])
    stat=(look_forward.new_cases.mean()-look_backward.new_cases.mean())/look_backward.new_cases.mean()
    if plot:
        plot_effect(rally=view_span,rally_date=selection[0],show_off=20,stat=stat)
    if verbose:
        print('county_state of selected rally: {}'.format(selection[1]))
        print('date of selected rally: {:%Y-%m-%d}'.format(selection[0]))
        print('the average number of new cases per day increased by: {:.02f} percent'.format(stat*100))
    if stat:
        stat_=[stat, selection[0], selection[1]]
    else:
        stat_=None
    return stat_

#### Trump Rallies with by mean percentage daily increase

In [ ]:
lst=[]
for i in range(21):
    lst.append([i]+trump_rally_effect(i,stat=True))
tmp=pd.DataFrame(lst, columns=['idx','mean_pct_daily_increase','date','county_state']).set_index('idx')
tmp['mean_pct_daily_increase']=np.round(tmp.mean_pct_daily_increase*100,2)
tmp=tmp.sort_values(by='mean_pct_daily_increase',ascending=False)
tmp

#### Impact of Trump Rallies | per pct increase in daily avg of cases 

In [ ]:
trump_rally_effect(7,plot=True);
trump_rally_effect(8,plot=True);
trump_rally_effect(5,plot=True);
trump_rally_effect(20,plot=True);
trump_rally_effect(19,plot=True);
trump_rally_effect(18,plot=True);
trump_rally_effect(1,plot=True);
trump_rally_effect(17,plot=True);
trump_rally_effect(14,plot=True);
trump_rally_effect(0,plot=True);

#### US by County | % change in cases over period of rally

In [ ]:
def plot_intra_state_distribution(df):
    plt.figure(figsize=(15, 4))
    plt.xticks(rotation=90)
    plt.xlabel('daily pct increase in cases')
    lb_lf=df.sort_values('stat')
    barlist=plt.bar(lb_lf.county_state_x,lb_lf.stat.replace([np.inf, -np.inf], np.nan))
    barlist[-8].set_color('r')

In [ ]:
def show_map(df,stat,center,hov='county_state',loc=None):
    
    """
    stat: name of column with stat
    center: list of lat lon of rally to center map on
    hov: column name of msg to display in hover tooltip
    loc: name of rally by county_state
    """
    
    fig = px.choropleth(df, geojson=counties, locations='fips', color=stat,
                               color_continuous_scale="Blues",
                               range_color=(0, max(df[stat])),
                               scope='usa',
                               hover_name=hov,
                               labels={stat:'mean change in new cases'},
                               center={'lat': center[0] , 'lon': center[1]}
                              )
    fig.update_geos(fitbounds="locations", visible=False)
                        
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
#    ax.annotate(‘Source: London Datastore, 2014’,xy=(0.1, .08),  
 #   xycoords='figure fraction', horizontalalignment='left', verticalalignment='top', fontsize=12, color='#555555')
    fig.show()

In [ ]:
def show_counties(df,idx):
    state=tmp.county_state[idx].split('_')[1]
    print(tmp.county_state[idx])
    look_forward=filter_time(df,tmp.date[idx],20,True)
    look_backward=filter_time(df,tmp.date[idx],20,False)
    lf=look_forward.query('state == "{}"'.format(state)).groupby(['county_state','fips'])[['new_cases']].mean().reset_index()
    lb=look_backward.query('state == "{}"'.format(state)).groupby(['county_state','fips'])[['new_cases']].mean().reset_index()
    lb_lf=lb.merge(lf, on='fips').merge(county_codes, on='fips')
    lb_lf['stat']=((lb_lf.new_cases_y-lb_lf.new_cases_x)/lb_lf.new_cases_x)*100
    #center=lb_lf.query('county_state_x == "{}"'.format(tmp.loc[idx].county_state))['coord'].values[0]
    show_map(lb_lf,center=center,stat='stat',hov='county_state_x')
    plot_intra_state_distribution(lb_lf)

#### Beltrami Minnesota

In [ ]:
# compare after-before
look_forward=filter_time(df,tmp.date[7],20,True)
look_backward=filter_time(df,tmp.date[7],20,False)
lf=look_forward.query('state == "Minnesota"').groupby(['county_state','fips'])[['new_cases']].mean().reset_index()
lb=look_backward.query('state == "Minnesota"').groupby(['county_state','fips'])[['new_cases']].mean().reset_index()
lb_lf=lb.merge(lf, on='fips')
lb_lf['stat']=((lb_lf.new_cases_y-lb_lf.new_cases_x)/lb_lf.new_cases_x)*100
#center=lb_lf.query('county_state_x == "{}"'.format(tmp.loc[8].county_state))['coord'].values[0]
#center=['{:.02f}'.format(i) for i in center]
show_map(lb_lf,center=center,stat='stat',hov='county_state_x',loc=tmp.county_state[7].split('_')[0])

#### Marathon Wisconsin

In [ ]:
show_counties(df,8)

#### Fulton Ohio

In [ ]:
show_counties(df,5)

#### Lackawanna Pennsylvania

In [ ]:
show_counties(df,15)

#### Creek Oklahoma

In [ ]:
show_counties(df,20)